Import Data

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import altair as alt
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col

session.sql("USE DATABASE EVENT").collect()
session.sql("USE SCHEMA DATATHON_2025_TEAM_KAPPA").collect()


In [ ]:
 # Load data from tables
race_df = session.table("VW_ALLRACESTATS")

In [ ]:
print("Race Table:")
race_df.show()

In [ ]:
race_pdf=race_df.to_pandas()

In [ ]:
race_pdf['RACE_DATE'].unique()

In [ ]:
#All data from 2013 and previous already deleted using sql


# # Filter out rows where 'YEAR' is less than or equal to 2013
# new_df = race_df.filter(race_df["YEAR"] > 2013)

# # Show the resulting DataFrame
# new_df.show()

In [ ]:
race_pdf.describe()

In [ ]:
#check for nulls
race_pdf.isnull().sum()

In [ ]:
#Creating a subset of race_pdf dataframe to show missing values
race_pdf_missing_value = race_pdf[race_pdf.isna().any(axis=1)]
race_pdf_missing_value.shape

#Inspecting the entries with missing values

race_pdf_missing_value.head(race_pdf_missing_value.shape[0])

In [ ]:
constructor_pdf= race_pdf.set_index('CONSTRUCTOR_NAME')
constructor_pdf.head()

In [ ]:
constructor_pdf.duplicated().sum()
constructor_pdf[constructor_pdf.duplicated()]

In [ ]:
st.write("Number of laps by Driver Name")
laps_by_winner = constructor_pdf.groupby("DRIVER_NAME")["LAPS_COMPLETED"].sum()
st.bar_chart(laps_by_winner)

In [ ]:
#Number of wins by constructor
st.write("Number of wins by Constructor Name")
wins_by_constructor = constructor_pdf.groupby("CONSTRUCTOR_NAME")["WINS"].sum()
st.bar_chart(wins_by_constructor)

In [ ]:
#Relationship between how many laps completed and how many wins.
st.write("Number of laps by wins")
laps_by_winner = constructor_pdf.groupby("LAPS_COMPLETED")["WINS"].sum()
st.bar_chart(laps_by_winner)

In [ ]:
# Filter to only the **winning constructor** per race (RACE_POSITION == 1)
winners_df = constructor_pdf[
    (constructor_pdf["RACE_POSITION_ORDER"] >= 1) & 
    (constructor_pdf["RACE_POSITION_ORDER"] <= 3)
]
print(winners_df.head())

In [ ]:
winners_df.head()

In [ ]:
# winners_pdf = winners_df.to_pandas()
# winners_pdf= winners_pdf.set_index('l_0006_CONSTRUCTOR_NAME')
# winners_pdf.head(20)

In [ ]:
st.write("Number of laps by wins")
laps_by_winner = winners_df.groupby("LAPS_COMPLETED")["WINS"].sum()
st.bar_chart(laps_by_winner)

In [ ]:
winners_df.head()

In [ ]:
winners_df.columns

In [ ]:
#Grid position and wins
st.write("Number of wins by grid position")
win_by_gridposition = winners_df.groupby("GRID_POSITION")["WINS"].sum()
st.bar_chart(win_by_gridposition)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from snowflake.snowpark.functions import col

In [ ]:
winners_df.columns

In [ ]:
# #renaming columns for clarity
# winners_pdf.rename(columns={
#     'r_0007_CONSTRUCTOR_NAME':'CONSTRUCTOR_NAME',
#     # add others if needed
# }, inplace=True)

In [ ]:
# winners_pdf.head()

In [ ]:
# Print columns to debug
print(winners_df.columns.tolist())

#Create binary target (McLaren win)
winners_df["MCLAREN_WIN"] = ((winners_df["RACE_POSITION_ORDER"] == 1) &
                              (winners_df["CONSTRUCTORID"] == 1 )).astype(int)

In [ ]:
winners_df["MCLAREN_WIN"]

In [ ]:
mclaren_df = winners_df[['CONSTRUCTORID', 'MCLAREN_WIN']]
mclaren_df.head()

In [ ]:
mclaren_wins = mclaren_df.groupby(['CONSTRUCTORID'], as_index=False)['MCLAREN_WIN'].sum()

plt.figure(figsize=(12, 8))
plot_mg = sns.countplot('MCLAREN_WIN', data=mclaren_df)

plt.title("Number of wins by McLaren")
plt.ylabel("Win Count")

In [ ]:
winners_df.columns

In [ ]:
# Correlation matrix (numeric only)
numeric_cols = [
    "GRID_POSITION", "FASTESTLAP", "FASTEST_LAP_RANK", "FASTESTLAPSPEED",
    "LAPS_COMPLETED", "POINTS", "RACE_POSITION_ORDER"
]
plt.figure(figsize=(10, 6))
sns.heatmap(winners_df[numeric_cols].corr(), annot=True, cmap="coolwarm")
plt.title("Feature Correlation Matrix")
plt.tight_layout()
plt.show()

In [ ]:
# Distribution plots of key features
key_features = ["GRID_POSITION", "FASTESTLAP", "FASTEST_LAP_RANK", "FASTESTLAPSPEED"]

for col_name in key_features:
    plt.figure(figsize=(6, 3))
    sns.histplot(winners_df[col_name], kde=True, bins=30)
    plt.title(f"Distribution of {col_name}")
    plt.tight_layout()
    plt.show()

In [ ]:
# McLaren win frequency
plt.figure(figsize=(5, 3))
sns.countplot(x="MCLAREN_WIN", data=winners_df)
plt.title("McLaren Wins vs Non-Wins")
plt.xticks([0, 1], ["No Win", "Win"])
plt.tight_layout()
plt.show()

In [ ]:
#Compare features vs. target (McLaren Win)
import seaborn as sns
import matplotlib.pyplot as plt

for col_name in key_features:
    plt.figure(figsize=(6, 3))
    sns.violinplot(x="MCLAREN_WIN", y=col_name, data=winners_df)
    plt.title(f"{col_name} Distribution vs. McLaren Win")
    plt.xticks([0, 1], ["No", "Yes"])
    plt.tight_layout()
    plt.show()

In [ ]:
# Compare mean of each feature for McLaren win vs non-win
group_stats = winners_df.groupby("MCLAREN_WIN")[key_features].mean().T
group_stats.columns = ["No Win", "Win"]
print("Feature averages by McLaren win:\n", group_stats)

In [ ]:
# Which constructors win most?
winners = winners_df[winners_df["RACE_POSITION_ORDER"] == 1]
plt.figure(figsize=(10, 4))
sns.countplot(y="CONSTRUCTORID", data=winners, order=winners["CONSTRUCTORID"].value_counts().index)
plt.title("Constructor Win Frequency")
plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(winners_df[["MCLAREN_WIN", "GRID_POSITION", "POINTS", "FASTEST_LAP_RANK"]], hue="MCLAREN_WIN")
plt.show()

In [ ]:
#Finding all unique values in Year column
# winners_df['YEAR'].unique()

In [ ]:
winners_df.head(2)

In [ ]:
mclaren_won_year = winners_df[
    (winners_df["MCLAREN_WIN"] != 0)
]

print(mclaren_won_year[["RACE_DATE", "MCLAREN_WIN"]])

In [ ]:
#Create binary target (Mercedes win) to compare McLaren and Mercedes wins
winners_df["MERCEDES_WIN"] = ((winners_df["RACE_POSITION_ORDER"] == 1) &
                              (winners_df["CONSTRUCTORID"] == 131)).astype(int)
# Print columns to debug
print(winners_df.columns.tolist())

In [ ]:
#FInding all dates where Mercedes won
mercedes_won_year = winners_df[
    (winners_df["MERCEDES_WIN"] != 0)
]

print(mercedes_won_year[["RACE_DATE", "MERCEDES_WIN"]])

In [ ]:
#Mercedes Grid position and wins
st.write("Mercedes: Number of wins by grid position")
mercedes_win_by_gridposition = winners_df.groupby("GRID_POSITION")["MERCEDES_WIN"].sum()
st.bar_chart(mercedes_win_by_gridposition)

In [ ]:
#McLaren dataset
# 3. CREATE TARGET COLUMN (McLaren in top 3)
winners_df["MCLAREN_TOP3"] = ((winners_df["CONSTRUCTOR_NAME"] == "McLaren") &
                      (winners_df["RACE_POSITION_ORDER"].between(1, 3))).astype(int)



In [ ]:
# Position gain/loss
df["POSITION_DELTA"] = df["GRID_POSITION"] - df["RACE_POSITION_ORDER"]

# McLaren flag
df["IS_MCLAREN"] = (df["CONSTRUCTOR_NAME"] == "McLaren").astype(int)

# McLaren top 3 finish target
df["MCLAREN_TOP3"] = ((df["IS_MCLAREN"] == 1) & (df["RACE_POSITION_ORDER"] <= 3)).astype(int)

# Sort by date for rolling features
df = df.sort_values(["DRIVERID", "RACE_DATE"])

# Rolling average for driver's recent performance (last 3 races)
df["DRIVER_AVG_POSITION"] = df.groupby("DRIVERID")["RACE_POSITION_ORDER"].transform(lambda x: x.shift(1).rolling(3).mean())

# Rolling average for constructor performance
df["TEAM_AVG_POSITION"] = df.groupby("CONSTRUCTOR_NAME")["RACE_POSITION_ORDER"].transform(lambda x: x.shift(1).rolling(3).mean())

In [ ]:
# Selecting features and target
X = winners_df[[
    "GRID_POSITION", "FASTESTLAP", "FASTEST_LAP_RANK", "FASTESTLAPSPEED",
    "LAPS_COMPLETED", "POINTS", "RACE_POSITION_ORDER"
]]
y = winners_df["MCLAREN_WIN"]

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# Scale features (optional, good for EDA + RF)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Split into training and testing
# X_train, X_test, y_train, y_test = train_test_split(
#     X_scaled, y, test_size=0.2, random_state=42
# )